In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json
import pickle

In [7]:
df = pd.read_csv('/content/drive/MyDrive/집교 2_Team P/user-track-listen_count_filtered5.csv')

In [8]:
# Pickle 파일 읽기
with open('/content/drive/MyDrive/집교 2_Team P/lyrics_Embedding/all_embeddings_emotion_full.pkl', 'rb') as file:
    data = pickle.load(file)

# DataFrame으로 변환
df_embedding = pd.DataFrame(data, columns=['embedding', 'track_id'])

# track_id를 정수로 변환 (필요하다면)
df_embedding['track_id'] = df_embedding['track_id'].astype(int)

# 'embedding' 열을 768차원의 각 차원으로 나누기
# df_embedding[['embedding_{}'.format(i) for i in range(768)]] = pd.DataFrame(df_embedding['embedding'].tolist(), index=df_embedding.index)

# 'embedding' 열 삭제
# df_embedding = df_embedding.drop(['embedding'], axis=1)

# DataFrame 확인
print(df_embedding.head())


                                           embedding  track_id
0  [-0.7747855, -0.4393492, 0.8827976, 0.5041653,...         2
1  [-0.05353763, -0.3497014, 0.41607994, 0.289434...         8
2  [-0.7386549, -0.53442806, 0.3141608, 0.384558,...      1524
3  [-0.35336906, 0.3955285, 0.1589705, -0.5695985...      1785
4  [-0.3986652, -0.5083513, 0.81219405, -0.062965...      1787


In [9]:
print(df.shape)
df = pd.merge(df, df_embedding, on='track_id', how='inner')
df.shape

(4645010, 4)


(4644051, 5)

In [10]:
df.head()

,Unnamed: 0,user_id,track_id,listen_count_bin,embedding
0,0,21072247,2,5,"[-0.7747855, -0.4393492, 0.8827976, 0.5041653,..."
1,1,32226961,2,5,"[-0.7747855, -0.4393492, 0.8827976, 0.5041653,..."
2,2,2115525,2,5,"[-0.7747855, -0.4393492, 0.8827976, 0.5041653,..."
3,3,4812967,2,3,"[-0.7747855, -0.4393492, 0.8827976, 0.5041653,..."
4,4,21211675,2,1,"[-0.7747855, -0.4393492, 0.8827976, 0.5041653,..."


In [11]:
from sklearn.preprocessing import LabelEncoder
user_encoder = LabelEncoder()
track_encoder = LabelEncoder()
lyrics_encoder = LabelEncoder()
df['user_id'] = user_encoder.fit_transform(df['user_id'])
df['track_id'] = track_encoder.fit_transform(df['track_id'])
df_embedding['track_id'] = lyrics_encoder.fit_transform(df_embedding['track_id'])

In [12]:
lyrics_dict = dict(zip(df_embedding['track_id'], df_embedding['embedding']))

In [13]:
# !pip install torch torchvision -U

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from tqdm import tqdm

# 데이터 불러오기 (예시: CSV 파일)
# Label Encoding
# PyTorch DataLoader에 맞게 데이터 변환
def df_to_tensor(dataset):
    users = torch.tensor(dataset['user_id'].values, dtype=torch.int)
    items = torch.tensor(dataset['track_id'].values, dtype=torch.int)
    ratings = torch.tensor(dataset['listen_count_bin'].values, dtype=torch.float)
    lyrics_embeddings = torch.tensor(np.vstack(dataset['embedding'].values), dtype=torch.float)
    return users, items, ratings, lyrics_embeddings

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

train_users, train_items, train_ratings, train_lyrics_embeddings = df_to_tensor(train_df)
test_users, test_items, test_ratings,test_lyrics_embeddings = df_to_tensor(test_df)

train_data = TensorDataset(train_users, train_items, train_ratings,train_lyrics_embeddings)
test_data = TensorDataset(test_users, test_items, test_ratings,test_lyrics_embeddings)

train_loader = DataLoader(train_data, batch_size=256, shuffle=True)
test_loader = DataLoader(test_data, batch_size=256, shuffle=False)
num_users = (df['user_id'].nunique())
num_items = (df['track_id'].nunique())
print(num_users)
print(num_items)

23761
28309


In [14]:
# NCF 모델 정의
class NCF_embedding(nn.Module):
    def __init__(self, num_users, num_items, embedding_size):
        super(NCF_embedding, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_size)
        self.item_embedding = nn.Embedding(num_items, embedding_size)
        self.lyrics_embedding = nn.Linear(768, embedding_size)
        self.fc_layers = nn.Sequential(
            nn.Linear(embedding_size * 3, embedding_size),
            nn.ReLU(),
            nn.Linear(embedding_size, int(embedding_size/2)),
            nn.ReLU(),
            nn.Linear(int(embedding_size/2), int(embedding_size/4)),
            nn.ReLU(),
            nn.Linear(int(embedding_size/4), int(embedding_size/8)),
            nn.ReLU(),
            nn.Linear(int(embedding_size/8), 1)
        )

    def forward(self, user, item, lyrics_embedding):
        user_embedding = self.user_embedding(user)
        item_embedding = self.item_embedding(item)
        lyrics_embedding = self.lyrics_embedding(lyrics_embedding.reshape(lyrics_embedding.shape[0],768))
        x = torch.cat((user_embedding, item_embedding,lyrics_embedding), dim=1)
        x = self.fc_layers(x)
        return x

# CUDA 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
def train_embedding(embedding,n_epoch):
  # NCF 모델 정의 및 GPU로 이동
  model = NCF_embedding(num_users=num_users, num_items=num_items, embedding_size=embedding)
  model.to(device)
  criterion = nn.MSELoss()
  optimizer = optim.Adam(model.parameters(), lr=0.001)
  # tqdm을 사용하여 학습 및 테스트 진행 상황 확인
  num_epochs = n_epoch
  for epoch in range(num_epochs):
      model.train()
      total_loss = 0
      for user, item, rating,lyrics_embedding in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
          optimizer.zero_grad()
          user, item, rating,lyrics_embedding = user.to(device), item.to(device), rating.to(device),lyrics_embedding.to(device)  # GPU로 이동
          output = model(user, item,lyrics_embedding.float())
          loss = criterion(output, rating.unsqueeze(1))
          loss.backward()
          optimizer.step()
          total_loss += loss.item()

      avg_loss = total_loss / len(train_loader)
      print(f'Epoch {epoch+1}/{num_epochs}, Avg. Loss: {avg_loss:.4f}')

      # 각 에폭이 끝날 때마다 테스트 데이터에 대한 예측 수행
      model.eval()
      all_predictions = []
      with torch.no_grad():
          for user, item, _,lyrics_embedding in tqdm(test_loader, desc=f'Testing Epoch {epoch+1}'):
              user, item,lyrics_embedding = user.to(device), item.to(device),lyrics_embedding.to(device)  # GPU로 이동
              output = model(user, item,lyrics_embedding)
              all_predictions.append(output)

      # RMSE 계산
      predictions = torch.cat(all_predictions).squeeze().cpu().numpy()  # CPU로 이동 후 numpy로 변환
      rmse = np.sqrt(mean_squared_error(test_df['listen_count_bin'].values, predictions))
      print(f'Epoch {epoch+1}/{num_epochs}, RMSE on test set: {rmse}')


Using device: cuda


In [16]:
train_embedding(64,23)

Epoch 1/23: 100%|██████████| 14513/14513 [01:41<00:00, 143.08it/s]


Epoch 1/23, Avg. Loss: 1.4281


Testing Epoch 1: 100%|██████████| 3629/3629 [00:16<00:00, 219.77it/s]


Epoch 1/23, RMSE on test set: 1.1560078617752845


Epoch 2/23: 100%|██████████| 14513/14513 [01:44<00:00, 138.28it/s]


...

Epoch 22/23, RMSE on test set: 1.1373516195068192


Epoch 23/23: 100%|██████████| 14513/14513 [01:44<00:00, 138.93it/s]


Epoch 23/23, Avg. Loss: 1.0649


Testing Epoch 23: 100%|██████████| 3629/3629 [00:15<00:00, 231.81it/s]

Epoch 23/23, RMSE on test set: 1.1338100652031298


In [ ]:
train_embedding(256,15)

Epoch 1/15: 100%|██████████| 14513/14513 [01:53<00:00, 128.33it/s]


Epoch 1/15, Avg. Loss: 1.3967


Testing Epoch 1: 100%|██████████| 3629/3629 [00:16<00:00, 214.13it/s]


Epoch 1/15, RMSE on test set: 1.1576325349111407


Epoch 2/15: 100%|██████████| 14513/14513 [01:53<00:00, 127.33it/s]


...

Epoch 14/15, RMSE on test set: 1.1276138759599303


Epoch 15/15: 100%|██████████| 14513/14513 [01:52<00:00, 128.78it/s]


Epoch 15/15, Avg. Loss: 0.9862


Testing Epoch 15: 100%|██████████| 3629/3629 [00:17<00:00, 205.77it/s]

Epoch 15/15, RMSE on test set: 1.1250230902348224


In [ ]:
train_embedding(512,15)

Epoch 1/15: 100%|██████████| 14513/14513 [02:11<00:00, 110.53it/s]


Epoch 1/15, Avg. Loss: 1.3911


Testing Epoch 1: 100%|██████████| 3629/3629 [00:17<00:00, 208.80it/s]


Epoch 1/15, RMSE on test set: 1.1558132476277687


Epoch 2/15: 100%|██████████| 14513/14513 [02:13<00:00, 108.39it/s]


...

Epoch 13/15: 100%|██████████| 14513/14513 [02:12<00:00, 109.54it/s]


Epoch 13/15, Avg. Loss: 1.0478


Testing Epoch 13: 100%|██████████| 3629/3629 [00:16<00:00, 220.17it/s]


Epoch 13/15, RMSE on test set: 1.1164659013041196


Epoch 14/15:  81%|████████  | 11743/14513 [01:45<00:28, 97.54it/s]

In [ ]:
train_embedding(768,30)

Epoch 1/30: 100%|██████████| 14513/14513 [02:31<00:00, 95.95it/s]


Epoch 1/30, Avg. Loss: 1.3889


Testing Epoch 1: 100%|██████████| 3629/3629 [00:16<00:00, 213.71it/s]


Epoch 1/30, RMSE on test set: 1.1552463778489663


Epoch 2/30: 100%|██████████| 14513/14513 [02:34<00:00, 93.70it/s]


...

Epoch 21/30, Avg. Loss: 0.8756


Testing Epoch 21: 100%|██████████| 3629/3629 [00:17<00:00, 209.38it/s]


Epoch 21/30, RMSE on test set: 1.1296161741798514


Epoch 22/30:   8%|▊         | 1178/14513 [00:12<02:26, 91.17it/s]


KeyboardInterrupt: ignored